In [5]:
from calc_self_corr import *
from machine_lib_output import *

In [ ]:

username = ""
password = ""

# 登录
sess = sign_in(username, password)

# 增量下载数据
download_data(sess, flag_increment=True)

# 加载数据， 如果需要使用不同的标签，可以传入 tag 参数， 例如 tag='PPAC' 或 tag='SelfCorr'
os_alpha_ids, os_alpha_rets = load_data()

从偏移 0 到 30 获取 alpha
新下载的 alpha 数量: 1, 总 alpha 数量: 149


In [7]:
ppa_tracker = get_alphas_from_csv("output/simulated_alphas.csv.2025-06-07", 1, 0.3, "submit")
## 将get的alpha的id取出至stone_bag
# 将 fail 的去掉了
stone_bag = []
for alpha in ppa_tracker:
    stone_bag.append(alpha[0])
print(stone_bag)    
print(len(stone_bag))

['LEvGPle', '29gm8AJ', 'mdG6JY9', 'VZrdK1w', 'xVL1PwW', 'r9qREdm', 'XV39Pv5', 'eNY5OLd', 'ORLW6Xd', 'w3dNz55', 'eNY1rbN', 'mdGdZzK', 'xVL20lp', '31K51j0', 'nmjrbaE', 'zgxEJVK', '5rmmR8o', '0xVJG1r', 'nmjL9NE', 'oRd1Q5l', '6Zk8OMP', '6ZkVvP5', 'w3rx6qp', 'ENdW6nG', '73p9kbv', 'Jo9Kmvj', 'jo796V5', 'MdrkWzk', '6ZkEWEP', 'w3rEqed', '8VwP1Jm', 'gX2eNLJ', '5rXJaak', 'OR3W9l1', 'GXRKzq5', 'oRdbLGv', 'lEmmKQ8', 'W35vovO', '318qQWg', '1OrmjgR', '6Z5RPl5', 'bJeQkOq', '73ezdzx', 'gXn8eoe', '318l8OX', 'ZgObrb8', 'nmLd35q', 'vZW2NZw', 'ORlW9eq', '98NGpKo', 'PxkxAZW', 'qOQwv1E', 'KvWMn9x', 'AeooZRR', 'm0PLZwK', 'vpKRj2d', 'NJX03rX', 'pMqXaMV', 'VmvjKKV', 'QN1LKVX', 'z2va81E', 'WoEnLNk', 'LmPlqQn', 'm0PrwGK', 'LmPg66n', 'OM1o0zb', 'NJXVb9L', 'kWxNlpO', 'PYJo6MK', 'OM1WeNJ', 'K5Kv0b8', 'x2K2Y8w', 'eVlG8lz', 'd32zlGg', 'x2KW5OW', '0Nb5ALp']
76


In [8]:
# 初始化一个空列表来存储符合条件的 alpha_id 和 self_corr
valid_alphas = []
idx = 0

# 遍历 stone_bag 中的每个 alpha_id
for alpha_id in stone_bag:
    idx += 1
    if len(stone_bag) > 1 and (idx % max(1, int(len(stone_bag) * 0.1)) == 0 or idx == 1 or idx == len(stone_bag)):
        # 检查 username 和 password 是否为 None
        if username is not None and password is not None:
            if not (idx == len(stone_bag) or len(stone_bag) < 100):
                new_sess = sign_in(username, password)
                if new_sess is None:
                    print("Failed to re-authenticate session, continuing with existing session")
                else:
                    sess = new_sess
                    print("Session re-authenticated successfully")
        else:
            print("Username or password is None, skipping re-authentication and continuing with existing session")

        
        print(
            f"Processing alpha {idx}/{len(stone_bag)} ({idx/len(stone_bag):.1%}) | "
            f"Current: {alpha_id}"
        )
    os_alpha_ids, os_alpha_rets = load_data(tag='PPAC')  #SelfCorr 如果需要用到自相关则将 PPAC 改成 SelfCorr 即可
    # 计算自相关性并获取结果
    self_corr = calc_self_corr(
        sess=sess,
        alpha_id=alpha_id,
        os_alpha_rets=os_alpha_rets,
        os_alpha_ids=os_alpha_ids,
    )
    # 判断自相关性是否小于 0.5
    if self_corr < 0.5:
        # 如果符合条件，添加到列表中
        valid_alphas.append((alpha_id, self_corr))
    else:
        print(f"Excluded alpha {alpha_id} with self_corr {self_corr}")

# 打印出符合条件的 alpha_id 和 self_corr
for alpha_id, self_corr in valid_alphas:
    print(f"Alpha ID: {alpha_id}, Self Correlation: {self_corr}")

Processing alpha 1/76 (1.3%) | Current: LEvGPle
1lR0PxW    0.2477
aQ1w581    0.1590
Lmw3rQm    0.1037
NzNvXog    0.0544
eVzxlnN    0.0320
w2aWZ5Y    0.0281
w2jeZav    0.0192
OMj2gNv    0.0063
JWQWqv2    0.0038
6oQN615   -0.0102
x2b1bmp   -0.0380
OM7xPAv   -0.0392
1lx8a7Q   -0.0557
Mewv38L   -0.1170
7v65jr1   -0.1277
dtype: float64
Lmw3rQm    0.3457
OMj2gNv    0.2781
6oQN615    0.1654
OM7xPAv    0.0568
NzNvXog    0.0552
Mewv38L    0.0537
eVzxlnN    0.0441
7v65jr1    0.0424
1lR0PxW    0.0214
JWQWqv2    0.0195
w2aWZ5Y    0.0012
1lx8a7Q   -0.0301
aQ1w581   -0.0460
x2b1bmp   -0.0552
w2jeZav   -0.0914
dtype: float64
1lR0PxW    0.3569
aQ1w581    0.2313
w2aWZ5Y    0.1955
1lx8a7Q    0.1526
Lmw3rQm    0.1234
NzNvXog    0.1105
eVzxlnN    0.1017
6oQN615    0.0829
w2jeZav    0.0607
OMj2gNv    0.0069
x2b1bmp   -0.0019
JWQWqv2   -0.0253
7v65jr1   -0.0402
Mewv38L   -0.1348
OM7xPAv   -0.1426
dtype: float64
6oQN615    0.5656
OMj2gNv    0.2769
Lmw3rQm    0.2388
Mewv38L    0.1477
7v65jr1    0.0976
OM7xPAv

In [ ]:
# 打印可提交的alpha信息并按margin排序，在网页上找到alpha手动提交
view_alphas_margin(valid_alphas)